In [32]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///new_mlflow.db"

client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)

In [33]:
experiments = client.search_experiments()
for experiment in experiments:
    print(experiment)

<Experiment: artifact_location='/home/ganesh/Documents/mlops_zoomcamp/mlops-zoomcamp/02-Experiment-tracking/mlruns/2', creation_time=1717004701793, experiment_id='2', last_update_time=1717004701793, lifecycle_stage='active', name='testing_experiment', tags={}>
<Experiment: artifact_location='/home/ganesh/Documents/mlops_zoomcamp/mlops-zoomcamp/02-Experiment-tracking/mlruns/1', creation_time=1716913801964, experiment_id='1', last_update_time=1716913801964, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>
<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716913756668, experiment_id='0', last_update_time=1716913756668, lifecycle_stage='active', name='Default', tags={}>


In [ ]:
client.create_experiment("testing_experiment")

In [16]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids="1",
    filter_string="metrics.rmse < 5.3",
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results= 5,
    order_by=["metrics.rmse ASC"]
)

In [17]:
for run in runs:
    print(f"run id : {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id : 41132515eb5149bca0c3504f1c81cf71, rmse: 5.2037
run id : 5ebd753769d54e79be8f5971a840e7a8, rmse: 5.2037
run id : f9132dfdb6b249e78d1d98457e262158, rmse: 5.2037
run id : 3cbc100dfe9a404aaa6f95a3e0f31465, rmse: 5.2095
run id : adf5668bfba74375b7f52358ef3c99d1, rmse: 5.2176


In [18]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [64]:
run_id = "41132515eb5149bca0c3504f1c81cf71"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="taxi-regressor")

Registered model 'taxi-regressor' already exists. Creating a new version of this model...
Created version '5' of model 'taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1717008236535, current_stage='None', description=None, last_updated_timestamp=1717008236535, name='taxi-regressor', run_id='41132515eb5149bca0c3504f1c81cf71', run_link=None, source='/home/ganesh/Documents/mlops_zoomcamp/mlops-zoomcamp/02-Experiment-tracking/mlruns/1/41132515eb5149bca0c3504f1c81cf71/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>

In [65]:
model_uri

'runs:/41132515eb5149bca0c3504f1c81cf71/model'

In [66]:
model_name = "taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Production
version: 4, stage: Staging
version: 5, stage: None


/tmp/ipykernel_123076/1670613113.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [68]:
client.transition_model_version_stage(
    name=model_name,
    version=5,
    stage="staging",
    archive_existing_versions=False
)

/tmp/ipykernel_123076/3734300119.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1717008236535, current_stage='Staging', description=None, last_updated_timestamp=1717008263576, name='taxi-regressor', run_id='41132515eb5149bca0c3504f1c81cf71', run_link=None, source='/home/ganesh/Documents/mlops_zoomcamp/mlops-zoomcamp/02-Experiment-tracking/mlruns/1/41132515eb5149bca0c3504f1c81cf71/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>

In [54]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=4,
    description=f"Updated the model version 4 to staging on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1717007699985, current_stage='Staging', description='Updated the model version 4 to staging on 2024-05-30', last_updated_timestamp=1717007741924, name='taxi-regressor', run_id='34bc2c58446a4cd0af5089fc1ac1d6e6', run_link=None, source='/home/ganesh/Documents/mlops_zoomcamp/mlops-zoomcamp/02-Experiment-tracking/mlruns/1/34bc2c58446a4cd0af5089fc1ac1d6e6/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [55]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID','DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [56]:
df = read_dataframe("./green_data/green_tripdata_2023-03.parquet")

In [69]:
client.download_artifacts(run_id=run_id, path="models_mlflow", dst_path=".")

'/home/ganesh/Documents/mlops_zoomcamp/mlops-zoomcamp/02-Experiment-tracking/models_mlflow'

In [71]:
import pickle

with open("./models_mlflow/model.xgb", "rb") as f_in:
    dv = pickle.load(f_in)

UnpicklingError: unpickling stack underflow

In [63]:
X_test = preprocess(df, dv)

AttributeError: 'RandomForestRegressor' object has no attribute 'transform'